# Notebook to process the injected waveforms and get detection statistics

In [2]:
import sys
sys.path.append('../tools')
from plot_tools import *
from map_tools import *
from params import *

import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'holoviews'

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
mapper = joblib.load('../tools/mapper.sav')
kde = joblib.load('../tools/kde_0_3.sav')
curves = joblib.load('../tools/contours_kde.sav')

Set the injection type and injection set (made several rounds of injections with different parameters, the ones relevant now are all_sky_based and no_constraints)

In [4]:
# inj_type = 'sg'
# inj_type = 'rd'
# inj_type = 'ga'
# inj_type = 'cg'
inj_type = 'cg_inc'
# inj_type = 'cg_double'
# inj_type = 'wn'

# inj_set = ''
inj_set = 'all_sky_based'
# inj_set = 'backup'
# inj_set = 'no_constraints'

data_path = join(features_path, 'fromraw_' + condition_method + '/' + model + '/H1/injected/' + inj_set + '/' + inj_type)
files = [f for f in sorted(listdir(data_path)) if isfile(join(data_path, f)) and f.split('.')[-1] == 'hdf5']
print(len(files))

20


In [5]:
def parse_file_name(file):
    """Function to parse injection file name
    """
    f0 = np.nan
    Q = np.nan
    tau = np.nan
    str_list = file.split('.')[0].split('-')
    inj_type = str_list[1]
    A = float('-'.join(str_list[3:5]))
    if inj_type == 'sg' or inj_type == 'cg' or inj_type == 'cg_inc' or inj_type == 'cg_double':
        f0 = int(str_list[6])
        Q = int(str_list[-1])
    
    elif inj_type == 'rd':
        f0 = int(str_list[6])
        tau = float('-'.join(str_list[-2:]))
    
    elif inj_type == 'ga':
        tau = float('-'.join(str_list[-2:]))
    
    elif inj_type == 'wn':
        f0 = int(str_list[6])
        tau = float('-'.join(str_list[-2:]))
    
    return inj_type, A, f0, Q, tau

In [6]:
x_examples, features_examples, y_examples, times_examples = load_train_examples(condition_method, model, features_path)
labels = np.unique(y_examples)

umap_examples = mapper.transform(features_examples)
tomap = {'ex': {'x': x_examples, 'y': y_examples, 'umap': umap_examples, 'times': times_examples}}

/anaconda2/envs/py37/lib/python3.7/site-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../../../anaconda2/envs/py37/lib/python3.7/site-packages/umap/nndescent.py", line 124:
    @numba.njit(parallel=True)
    def init_from_random(n_neighbors, data, query_points, heap, rng_state):
    ^

  state.func_ir.loc))
/anaconda2/envs/py37/lib/python3.7/site-packages/numba/core/typed_passes.py:314: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../../../../anaconda2/envs

Load map space representations and deviation scores of the injections, and compute their kde_scores

In [7]:
times_tot = []
umap_tot = {'H': [], 'L': []}
y_hat_tot = {'H': [], 'L': []}
kde_scores = {'H': [], 'L': []}
devs = {'H': [], 'L': []}

inj_type_tot = []
A_tot = []
f0_tot = []
Q_tot = []
tau_tot = []
num_points = 0

for num in range(len(files)):
    file = files[num]
    
    for detector in ['H', 'L']:
        data_path = join(features_path, 'fromraw_' + condition_method + '/' + model + '/' + detector + '1/injected/' + inj_set + '/' + inj_type)
        
        with h5py.File(join(data_path, file), 'r') as f:
            umap = np.asarray(f['umap'])
            times = np.asarray(f['times'])
            y_hat = np.asarray(f['y_hat'])
            total_dev = np.asarray(f['total_dev'])
        
        pred_idx = np.argmax(y_hat, axis=1)
        umap_tot[detector].extend(umap)
        y_hat_tot[detector].extend(labels[pred_idx])
        kde_scores[detector].extend(kde.score_samples(umap))
        devs[detector].extend(total_dev)
        if detector == 'H':
            times_tot.extend(times)
        
    num_points += len(times)
    inj_type, A, f0, Q, tau = parse_file_name(file)
    inj_type_tot.extend([inj_type] * len(times))
    A_tot.extend([A] * len(times))
    f0_tot.extend([f0] * len(times))
    Q_tot.extend([Q] * len(times))
    tau_tot.extend([tau] * len(times))

for key in umap_tot.keys():
    umap_tot[key] = np.asarray(umap_tot[key])
    
print(num_points) #, len(times_tot), len(inj_type_tot), len(kde_scores['H']), len(devs['L']))
print(umap_tot['H'].shape, len(y_hat_tot['H']))

df = pd.DataFrame({'time': times_tot, 'inj_type': inj_type_tot, 'A': A_tot, 'f0': f0_tot, 
                   'Q': Q_tot, 'tau': tau_tot, 'H_kde': kde_scores['H'], 
                   'L_kde': kde_scores['L'], 'H_dev': devs['H'], 'L_dev': devs['L'], 
                   'H_pred': y_hat_tot['H'], 'L_pred': y_hat_tot['L']})
display(df.head())

300
(300, 2) 300


,time,inj_type,A,f0,Q,tau,H_kde,L_kde,H_dev,L_dev,H_pred,L_pred
0,1.127032e+09,cg_inc,1.500000e-21,30,100,NaN,-5.714124,-5.223252,1507.507693,3321.130842,Whistle,Whistle
1,1.136360e+09,cg_inc,1.500000e-21,30,100,NaN,-5.183068,-5.659626,405.024337,550.483821,Whistle,Whistle
2,1.126092e+09,cg_inc,1.500000e-21,30,100,NaN,-5.463617,-5.515406,1870.203443,2787.843011,Whistle,Whistle
3,1.126581e+09,cg_inc,1.500000e-21,30,100,NaN,-5.287364,-5.072514,1094.383670,1322.501591,Whistle,Whistle
4,1.126651e+09,cg_inc,1.500000e-21,30,100,NaN,-5.495837,-5.621832,985.024368,1471.567848,Whistle,Whistle


Print the number of injections classified as 'None_of_the_Above'. Not actually used, but the information is nice to have

In [8]:
print(sum(df['H_pred'] == 'None_of_the_Above'))
print(sum(df['L_pred'] == 'None_of_the_Above'))

9
8


Get all injections that are flagged by either method

In [9]:
dev_th = gram_th

df_ood = df.loc[(df['H_kde'] < kde_th) | (df['L_kde'] < kde_th) | (df['H_dev'] > dev_th) | (df['L_dev'] > dev_th)]
# df_ood = df.loc[((df['H_kde'] < kde_th) | (df['L_kde'] < kde_th) | (df['H_dev'] > dev_th) | (df['L_dev'] > dev_th)) & (df['A'] == 6e-20)]
# df_ood = df.loc[((df['H_kde'] < kde_th) | (df['L_kde'] < kde_th)) & ((df['H_dev'] > dev_th) | (df['L_dev'] > dev_th))]
# df_ood = df.loc[(df['H_dev'] > dev_th) | (df['L_dev'] > dev_th)]
display(df_ood)
print(len(df_ood))

,time,inj_type,A,f0,Q,tau,H_kde,L_kde,H_dev,L_dev,H_pred,L_pred
30,1.127032e+09,cg_inc,2.900000e-21,30,100,NaN,-6.313153,-5.376054,5861.555805,5877.721070,Whistle,Whistle
32,1.126092e+09,cg_inc,2.900000e-21,30,100,NaN,-6.233884,-5.552873,5240.088096,5558.820171,Whistle,Whistle
38,1.135543e+09,cg_inc,2.900000e-21,30,100,NaN,-5.635748,-5.846911,5356.712932,5480.470723,Whistle,Whistle
39,1.126871e+09,cg_inc,2.900000e-21,30,100,NaN,-5.336054,-5.131463,4281.140286,6135.972508,Whistle,Chirp
41,1.126653e+09,cg_inc,2.900000e-21,30,100,NaN,-5.841775,-6.127519,5818.017600,4941.884916,Whistle,Whistle
...,...,...,...,...,...,...,...,...,...,...,...,...
226,1.136360e+09,cg_inc,2.940000e-20,30,9,NaN,-5.150903,-5.031303,7272.048233,2934.107157,Whistle,Whistle
229,1.126651e+09,cg_inc,2.940000e-20,30,9,NaN,-5.305894,-5.779257,5222.527716,6052.964703,Whistle,Whistle
232,1.126316e+09,cg_inc,2.940000e-20,30,9,NaN,-5.231082,-5.027551,5166.877320,2587.300964,Whistle,Whistle
234,1.126871e+09,cg_inc,2.940000e-20,30,9,NaN,-5.210687,-5.803041,5380.303660,1404.459774,Whistle,None_of_the_Above


67


Get number of detections for each set of parameters

In [10]:
params_path = Path('../../shared/injection_params')

H_kde_count = []
L_kde_count = []
both_kde_count = []
H_dev_count = []
L_dev_count = []
both_dev_count = []

if inj_type == 'wn':
    params = pd.read_csv(join(params_path, inj_type + '_params_csv.csv'), usecols=['f_low', 'f_high', 'tau', 'A'])
    for f0, tau, A in zip(params['f_low'], params['tau'], params['A']):
        H_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['H_kde'] < kde_th)))
        L_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['L_kde'] < kde_th)))
        both_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['H_kde'] < kde_th) & (df['L_kde'] < kde_th)))
        H_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['H_dev'] > dev_th)))
        L_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['L_dev'] > dev_th)))
        both_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (round(df['A'] * 1e22) == round(A * 1e22)) & (df['H_dev'] > dev_th) & (df['L_dev'] > dev_th)))

elif inj_type == 'rd':
    params = pd.read_csv(join(params_path, inj_type + '_params_csv.csv'), usecols=['f0', 'tau', 'A'])
    for f0, tau, A in zip(params['f0'], params['tau'], params['A']):
        H_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['H_kde'] < kde_th)))
        L_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['L_kde'] < kde_th)))
        both_kde_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['H_kde'] < kde_th) & (df['L_kde'] < kde_th)))
        H_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['H_dev'] > dev_th)))
        L_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['L_dev'] > dev_th)))
        both_dev_count.append(sum((df['f0'] == f0) & (df['tau'] == tau) & (df['A'] == A) & (df['H_dev'] > dev_th) & (df['L_dev'] > dev_th)))

elif inj_type == 'ga':
    params = pd.read_csv(join(params_path, inj_type + '_params_csv.csv'), usecols=['tau', 'A'])
    for tau, A in zip(params['tau'], params['A']):
        H_kde_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['H_kde'] < kde_th)))
        L_kde_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['L_kde'] < kde_th)))
        both_kde_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['H_kde'] < kde_th) & (df['L_kde'] < kde_th)))
        H_dev_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['H_dev'] > dev_th)))
        L_dev_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['L_dev'] > dev_th)))
        both_dev_count.append(sum((df['tau'] == tau) & (df['A'] == A) & (df['H_dev'] > dev_th) & (df['L_dev'] > dev_th)))

else:
    params = pd.read_csv(join(params_path, inj_type + '_params_csv.csv'), usecols=['f0', 'Q', 'A'])
    for f0, Q, A in zip(params['f0'], params['Q'], params['A']):
        H_kde_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['H_kde'] < kde_th)))
        L_kde_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['L_kde'] < kde_th)))
        both_kde_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['H_kde'] < kde_th) & (df['L_kde'] < kde_th)))
        H_dev_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['H_dev'] > dev_th)))
        L_dev_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['L_dev'] > dev_th)))
        both_dev_count.append(sum((df['f0'] == f0) & (df['Q'] == Q) & (df['A'] == A) & (df['H_dev'] > dev_th) & (df['L_dev'] > dev_th)))

params['H_kde_count'] = H_kde_count
params['L_kde_count'] = L_kde_count
params['both_kde_count'] = both_kde_count
params['H_dev_count'] = H_dev_count
params['L_dev_count'] = L_dev_count
params['both_dev_count'] = both_dev_count

if inj_set == 'no_constraints':
    params['H_dev_%'] = np.asarray(H_dev_count) * 100 / 1000
    params['L_dev_%'] = np.asarray(L_dev_count) * 100 / 1000
    params['both_dev_%'] = np.asarray(both_dev_count) * 100 / 1000

# params.to_csv(join(params_path, inj_type + '_ood_csv.csv'))
display(params)

,f0,Q,A,H_kde_count,L_kde_count,both_kde_count,H_dev_count,L_dev_count,both_dev_count
0,30,3,8.500000e-21,0,0,0,0,0,0
1,30,3,1.700000e-20,0,0,0,0,0,0
2,30,3,3.400000e-20,0,0,0,0,0,0
3,30,3,5.100000e-20,0,0,0,0,0,0
4,30,9,4.900000e-21,0,0,0,0,0,0
5,30,9,9.800000e-21,0,0,0,0,0,0
6,30,9,1.860000e-20,0,0,0,0,0,0
7,30,9,2.940000e-20,0,0,0,5,1,1
8,30,100,1.500000e-21,0,0,0,0,0,0
9,30,100,2.900000e-21,0,0,0,5,4,3


Load injection spectrograms

In [11]:
data_path = join(conditioned_path, detector + '1/' + condition_method + '/injected/' + inj_set + '/' + inj_type)
files = [f for f in sorted(listdir(data_path)) if isfile(join(data_path, f)) and f.split('.')[-1] == 'hdf5']

x_tot = {'H': [], 'L': []}

for num in range(len(files)):
    file = files[num]
    
    for detector in ['H', 'L']:
        data_path = join(conditioned_path, detector + '1/' + condition_method + '/injected/' + inj_set + '/' + inj_type)
        with h5py.File(join(data_path, file), 'r') as f:
            x = np.asarray(f['x'])
        x_tot[detector].extend(x)

for key in x_tot.keys():
    x_tot[key] = np.asarray(x_tot[key])
    print(x_tot[key].shape)

(300, 299, 299, 3)
(300, 299, 299, 3)


Set which injections to plot (either all, or just the ones that are flagged)

In [12]:
# plot_idx = 'all'
plot_idx = 'flagged'

if plot_idx == 'all':
    idx = df.index

elif plot_idx == 'flagged':
    idx = (df['H_kde'] < kde_th) | (df['L_kde'] < kde_th) | (df['H_dev'] > dev_th) | (df['L_dev'] > dev_th)

for detector in ['H', 'L']:
    tomap[detector] = {'x': x_tot[detector][idx], 'y': np.asarray(y_hat_tot[detector])[idx], 
                       'umap': umap_tot[detector][idx], 'times': np.asarray(times_tot)[idx], 
                       'total_dev': np.asarray(devs[detector])[idx], 
                       'inj_type': np.asarray(inj_type_tot)[idx], 
                       'A': np.asarray(A_tot)[idx], 'f0': np.asarray(f0_tot)[idx], 
                       'Q': np.asarray(Q_tot)[idx], 'tau': np.asarray(tau_tot)[idx]}

for key in tomap['H'].keys():
    print(key, tomap['H'][key].shape)

x (67, 299, 299, 3)
y (67,)
umap (67, 2)
times (67,)
total_dev (67,)
inj_type (67,)
A (67,)
f0 (67,)
Q (67,)
tau (67,)


Plot selected spectrograms

In [13]:
if plot_idx == 'all':
    layout = plot_both_dets(tomap, plot_ex=False, title='all ' + inj_type + ' injections (%d flagged)' % len(df_ood))

elif plot_idx == 'flagged':
    layout = plot_both_dets(tomap, index=df[idx].index, plot_ex=False, title=inj_type + ' injections flagged (%d flagged)' % len(df_ood))

points = layout.Points
H_dmap = layout.DynamicMap.I
L_dmap = layout.DynamicMap.II
width = 400
height = 400
newlay = (points.opts(toolbar='below') * curves + hv.Empty() 
        + H_dmap.opts(frame_width=width, frame_height=height) 
        + L_dmap.opts(frame_width=width, frame_height=height)).opts(merge_tools=False).cols(2)
newlay

:Layout
   .Overlay.I     :Overlay
      .Points.I   :Points   [feature_1,feature_2]   (times,y,marker,size,index)
      .Contours.I :Contours   [x,y]   (z)
   .Empty.I       :Empty
   .DynamicMap.I  :DynamicMap   []
      :RGB   [x,y]   (R,G,B)
   .DynamicMap.II :DynamicMap   []
      :RGB   [x,y]   (R,G,B)